In [ ]:
import pandas as pd
import numpy as np
import torch
import torchvision
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
from torch.utils.data import random_split
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

use_drive = True
if use_drive:
  PATH = "/content/drive/MyDrive/CIL 2022/"
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/My Drive/CIL 2022/
  !ls
else:
  PATH = "./"

Mounted at /content/drive
/content/drive/My Drive/CIL 2022
'basic urban dictionary+21. July RandomForestClassifier(n_estimators=128, random_state=0, n_jobs=-1)with-stemming_with-lemmatize_no-stopwords_with-spellcorrect.txt'
'basic urban dictionary+21. July RandomForestClassifier(n_estimators=128, random_state=0, n_jobs=-1)with-stemming_with-lemmatize_with-stopwords_no-spellcorrect.txt'
'Bert inspiration'
 BoW_v2.ipynb
 code
 data
 finetuned_roberta_model_in_steps
 finetuned_roberta_model_in_steps_submission.csv
 finetuned_roberta_model_in_steps_val.csv
 finetuned_roberta_model_in_steps_with_dropout
 finetuned_roberta_model_in_steps_with_dropout_half
 finetuned_robert_model2
 finetuned_robert_model2_test.csv
'For Euler Cluster'
'GRU4_stanford glove_raw_test.csv'
'GRU4_stanford glove_raw_train.csv'
'GRU4_stanford glove_raw_val.csv'
'Grubert stats and model'
'Grubert v.A.1.;epochnr=2 time_duraction=12342.342999219894s'
'Grubert v.A.1.;epochnr=3 time_duraction=12331.698219776154s'
'Grubert

In [ ]:
# HYPERPARAMETERS for data

TRUNCATE_TRAINING_DATA_TO = 500000 #200000 # number or -1 for no truncating

PREPROCESSING_OPTIONS = [ "raw",
"no-stemming_no-lemmatize_no-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_no-lemmatize_with-stopwords_with-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_no-spellcorrect",
"no-stemming_with-lemmatize_with-stopwords_with-spellcorrect",
"with-stemming_no-lemmatize_with-stopwords_no-spellcorrect",
"with-stemming_with-lemmatize_no-stopwords_with-spellcorrect",
"with-stemming_with-lemmatize_with-stopwords_no-spellcorrect" ]
PREPROCESSING_CHOICE = PREPROCESSING_OPTIONS[0] # one from PREPROCESSING_OPTIONS

pretrained_model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

In [ ]:
def read_file_and_strip(filename):
  lines = []
  with open(filename) as file:
    for line in file:
      lines.append(line.strip())
  return np.asarray(lines)

def read_data():
  dataset_path = PATH + "data/" + PREPROCESSING_CHOICE + "/"

  train_sentences = read_file_and_strip(dataset_path + "train_sentences.txt")
  train_labels = read_file_and_strip(dataset_path + "train_labels.txt").astype(int)
  val_sentences = read_file_and_strip(dataset_path + "val_sentences.txt")
  val_labels = read_file_and_strip(dataset_path + "val_labels.txt").astype(int)
  
  return train_sentences, train_labels, val_sentences, val_labels

X_train, y_train, X_test, y_test = read_data()
print(len(X_train))

if TRUNCATE_TRAINING_DATA_TO > 0:
  X_train = X_train[:TRUNCATE_TRAINING_DATA_TO]
  y_train = y_train[:TRUNCATE_TRAINING_DATA_TO]

print("len(X_train): %d" %len(X_train))
print("len(y_train): %d" %len(y_train))
print("len(X_test): %d" %len(X_test))
print("len(y_test): %d" %len(y_test))

2043433
len(X_train): 2043433
len(y_train): 2043433
len(X_test): 227049
len(y_test): 227049


In [ ]:
try:
  from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
  print("transformers package already installed")
except ModuleNotFoundError:
  ! pip install transformers datasets
  from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.6 MB/s 
     |████████████████████████████████| 365 kB 50.3 MB/s 
     |████████████████████████████████| 101 kB 11.3 MB/s 
     |████████████████████████████████| 596 kB 51.9 MB/s 
     |████████████████████████████████| 6.6 MB 30.9 MB/s 
     |████████████████████████████████| 212 kB 53.8 MB/s 
     |████████████████████████████████| 141 kB 53.6 MB/s 
     |████████████████████████████████| 127 kB 51.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [ ]:
from datasets import Dataset
train_dictionary = {"label": y_train, "text": X_train}
train_dataset = Dataset.from_dict(train_dictionary)
test_dictionary = {"label": y_test, "text": X_test}
test_dataset = Dataset.from_dict(test_dictionary)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

def tokenize_function(record):
    return tokenizer(record["text"], padding="max_length", max_length=75, truncation=True)

def tokenize_dataset(dataset):
  tokenized_datasets = dataset.map(tokenize_function, batched=True)
  tokenized_datasets = tokenized_datasets.remove_columns(["text"])
  tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
  print(tokenized_datasets)
  print(tokenized_datasets[0])
  tokenized_datasets.set_format("torch")
  return tokenized_datasets

train_tokenized_dataset = tokenize_dataset(train_dataset)
test_tokenized_dataset = tokenize_dataset(test_dataset)

  0%|          | 0/2044 [00:00<?, ?ba/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 2043433
})
{'labels': 0, 'input_ids': [0, 41552, 12105, 15698, 596, 24, 16, 98, 543, 7, 120, 5324, 17487, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


  0%|          | 0/228 [00:00<?, ?ba/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 227049
})
{'labels': 1, 'input_ids': [0, 354, 70, 59, 106, 12644, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_tokenized_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(test_tokenized_dataset, batch_size=8)

In [ ]:
# HYPERPARAMETERS for model

SHOULD_FREEZE_CLASSIFIER = False
REPLACE_LAST_LAYER = True
NUMBER_OF_NODES_FOR_LAST_LAYER = 2


LOAD_PRETRAINED_WEIGHTS_UNTIL_LAST_K_PARAMETERS = 4 # 2 or 4
if SHOULD_FREEZE_CLASSIFIER:
  LOAD_PRETRAINED_WEIGHTS_UNTIL_LAST_K_PARAMETERS = 2


REMAIN_THREE_LABELS = False
if REMAIN_THREE_LABELS:
  NUMBER_OF_NODES_FOR_LAST_LAYER = 3

ADD_ONE_EXTRA_LAYER = False

num_epochs = 5

lr = 1e-3

In [ ]:
model_with_weights = AutoModelForSequenceClassification.from_pretrained(pretrained_model_name)
print("↑ you can ignore above warning about weights not loaded.")

# if not REMAIN_THREE_LABELS:
#   model.num_labels = 2
print(model_with_weights.roberta.embeddings.word_embeddings.weight.data)

if not REMAIN_THREE_LABELS:
  config = AutoConfig.from_pretrained(pretrained_model_name)
  config.num_labels = 2
  # config.classifier_dropout = 0.2
  model = AutoModelForSequenceClassification.from_config(config)
  # we have to restore the weights, see https://stackoverflow.com/a/72940359/2529745
  print(model.roberta.embeddings.word_embeddings.weight.data)
  for parameter_name, parameter in list(model.named_parameters())[:-LOAD_PRETRAINED_WEIGHTS_UNTIL_LAST_K_PARAMETERS]:
    parameter.data = model_with_weights.state_dict().get(parameter_name).data.clone()
    print(parameter_name)
  
  print(model.roberta.embeddings.word_embeddings.weight.data)
else:
  model = model_with_weights


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


↑ you can ignore above warning about weights not loaded.
tensor([[ 0.1062, -0.0219,  0.1073,  ...,  0.0102, -0.0185, -0.0205],
        [-0.0098,  0.0382,  0.1892,  ...,  0.0786,  0.0935,  0.0373],
        [-0.0415, -0.0245, -0.0071,  ...,  0.0421, -0.0180, -0.0493],
        ...,
        [-0.0335,  0.0925,  0.0733,  ...,  0.0880,  0.0541,  0.0345],
        [-0.0108,  0.0454,  0.1191,  ...,  0.0080,  0.0899, -0.0048],
        [ 0.0478,  0.0302,  0.0396,  ..., -0.0031,  0.0100,  0.0257]])
tensor([[ 0.0118,  0.0003, -0.0148,  ...,  0.0005,  0.0198,  0.0140],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0191, -0.0022, -0.0125,  ..., -0.0180,  0.0031,  0.0029],
        ...,
        [-0.0077, -0.0389,  0.0049,  ...,  0.0100, -0.0019, -0.0152],
        [ 0.0218, -0.0086,  0.0076,  ...,  0.0175, -0.0364,  0.0192],
        [ 0.0135,  0.0323, -0.0178,  ..., -0.0059,  0.0259, -0.0521]])
roberta.embeddings.word_embeddings.weight
roberta.embeddings.position_embed

In [ ]:
# from tqdm import tqdm

# correct_guess = 0
# model.eval()
# X_test = np.asarray(["I hate this shit. It's the worst. Really bad, annoying, offensive. It just makes me sad", "This is great. I love how delicious this is. You are a wonderful cook. Congratulations."])
# y_test = np.asarray([0, 1])
# bar = tqdm(range(len(X_test)))
# for i in range(len(X_test)):
#   tweet, label = X_test[i], y_test[i]
#   print(tweet)
#   encoded_tweet = tokenizer(tweet, return_tensors='pt')
#   output = model(**encoded_tweet)
#   scores = output[0][0].detach().numpy()
#   scores = [scores[0], scores[2]]
#   print(scores)
#   label_to_choose = np.argmax(scores)
#   print(label_to_choose)
#   if label == label_to_choose:
#     correct_guess = correct_guess + 1
#   bar.update(1)

# print(correct_guess / len(X_test))
# model.train()

In [ ]:
for param in model.roberta.embeddings.parameters():
    param.requires_grad = False
for param in model.roberta.encoder.parameters():
    param.requires_grad = False
if SHOULD_FREEZE_CLASSIFIER:
  for param in list(model.classifier.parameters())[:-1]:
      param.requires_grad = False
      print(param)

if REPLACE_LAST_LAYER:
  model.classifier.out_proj = nn.Linear(in_features=768, out_features=NUMBER_OF_NODES_FOR_LAST_LAYER, bias=True)

if ADD_ONE_EXTRA_LAYER:
  model.classifier = nn.Sequential(
      model.classifier, 
      nn.Linear(in_features=NUMBER_OF_NODES_FOR_LAST_LAYER, out_features=2, bias=True)
  )

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to(device)

In [ ]:
print(model)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
from torch.optim import SGD, AdamW

optimizer = AdamW(model.parameters(), lr=lr, weight_decay=5e-2)
# optimizer = SGD(model.parameters(), lr=lr, momentum=0.9)

# from transformers.optimization import Adafactor
# # replace AdamW with Adafactor
# optimizer = Adafactor(
#     model.parameters(),
#     lr=lr,
#     eps=(1e-30, 1e-3),
#     clip_threshold=1.0,
#     decay_rate=-0.8,
#     beta1=None,
#     weight_decay=0.0,
#     relative_step=False,
#     scale_parameter=False,
#     warmup_init=False,
# )

In [ ]:
from datasets import load_metric
def evaluate_test(use_mini_dataset = False):
  acc_metric = load_metric("accuracy")
  f1_metric = load_metric("f1")
  model.eval()
  if not use_mini_dataset:
    batch_count = len(eval_dataloader)
    print("evaluation batch count %d" %batch_count)
    test_progress_bar = tqdm(range(batch_count))
  for i, batch in enumerate(eval_dataloader):
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = model(**batch)

      logits = outputs.logits
      if REMAIN_THREE_LABELS and not ADD_ONE_EXTRA_LAYER:
        scores = logits.cpu().numpy()
        removed_neutral_col = np.delete(scores, 1, axis=1)
        scores = torch.tensor(removed_neutral_col)
        predictions = torch.argmax(scores, dim=-1)
      else:
        predictions = torch.argmax(logits, dim=-1)
      acc_metric.add_batch(predictions=predictions, references=batch["labels"])
      f1_metric.add_batch(predictions=predictions, references=batch["labels"])
      if use_mini_dataset and i > 50:
        break
      if not use_mini_dataset:
        test_progress_bar.update(1)

  model.train()

  accuracy = acc_metric.compute()["accuracy"]
  if use_mini_dataset:
    return accuracy
  print({"accuracy": accuracy})
  print(f1_metric.compute())
  return accuracy

In [ ]:
from tqdm.auto import tqdm

In [ ]:
def evaluate_and_save_to_csv(model, dataloader, file_path, is_testset):
  acc_metric = load_metric("accuracy")
  f1_metric = load_metric("f1")
  model.eval()
  all_predictions = np.empty(0, dtype=int)

  batch_count = len(dataloader)
  print("evaluation batch count %d" %batch_count)
  test_progress_bar = tqdm(range(batch_count))
  for i, batch in enumerate(dataloader):
      batch = {k: v.to(device) for k, v in batch.items()}
      with torch.no_grad():
          outputs = model(**batch)

      logits = outputs.logits
      if REMAIN_THREE_LABELS and not ADD_ONE_EXTRA_LAYER:
        scores = logits.cpu().numpy()
        removed_neutral_col = np.delete(scores, 1, axis=1)
        scores = torch.tensor(removed_neutral_col)
        predictions = torch.argmax(scores, dim=-1)
      else:
        predictions = torch.argmax(logits, dim=-1)
      acc_metric.add_batch(predictions=predictions, references=batch["labels"])
      f1_metric.add_batch(predictions=predictions, references=batch["labels"])

      all_predictions = np.append(all_predictions, predictions.cpu().detach().numpy())
      
      test_progress_bar.update(1)

  if is_testset:
    for index in range(len(all_predictions)):
      if all_predictions[index] == 0:
        all_predictions[index] = -1

    # Create the pandas dataframe
    id = np.arange(1, len(all_predictions) + 1)
    data = {"Id": id, "Prediction": all_predictions}
    df = pd.DataFrame(data, columns=["Id", "Prediction"])
    #print(df)
    #save submission file
    df.to_csv(file_path, index=False)
    print("Submission file generated.")

  else:
    # np.savetxt(file_path, all_predictions, fmt="%d")
    df = pd.DataFrame(all_predictions)
    #print(df)
    #save submission file
    df.to_csv(file_path, index=False, header=None)

  model.train()

  accuracy = acc_metric.compute()["accuracy"]
  print({"accuracy": accuracy})
  print(f1_metric.compute())
  return accuracy

In [ ]:
model_name = "finetuned_roberta_model"
model = model.from_pretrained(PATH + "Saved Model States/finetuned_roberta_model")
model = model.to(device)
model.eval()

evaluate_path = PATH + "data/test data/"

In [ ]:
testset_data_path = "/content/drive/MyDrive/CIL 2022/data/test data/test_data.txt"

#read test data in
test_sentences = read_file_and_strip(testset_data_path)
print(len(test_sentences))
test_labels_list = [0] * len(test_sentences)
testtest_dictionary = {"label": test_labels_list, "text": test_sentences}
testtest_dataset = Dataset.from_dict(testtest_dictionary)
testtest_tokenized_dataset = tokenize_dataset(testtest_dataset)
testtest_dataloader = DataLoader(testtest_tokenized_dataset, shuffle=False, batch_size=8)

evaluate_and_save_to_csv(model, testtest_dataloader, evaluate_path + model_name + "_submission.csv", True)

10000


  0%|          | 0/10 [00:00<?, ?ba/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 10000
})
{'labels': 0, 'input_ids': [0, 134, 6, 16466, 109, 139, 1759, 3342, 2850, 15917, 36, 1612, 19, 5, 15295, 3342, 12, 417, 3036, 842, 8631, 22923, 4097, 935, 2156, 1095, 1181, 11, 5, 514, 8, 1666, 28696, 6423, 15698, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


evaluation batch count 1250


  0%|          | 0/1250 [00:00<?, ?it/s]

Submission file generated.
{'accuracy': 0.4957}
{'f1': 0.0}


0.4957

In [ ]:
val_acc = evaluate_and_save_to_csv(model, eval_dataloader, evaluate_path + model_name + "_val.csv", False)
print(f'\t Validation. Acc: {val_acc*100:.2f}%')

evaluation batch count 28382


  0%|          | 0/28382 [00:00<?, ?it/s]

In [ ]:
train_dataloader = DataLoader(train_tokenized_dataset, shuffle=False, batch_size=8)
train_acc = evaluate_and_save_to_csv(model, train_dataloader, evaluate_path + model_name + "_train.csv", False)
print(f'\t Training. Acc: {train_acc*100:.2f}%')

evaluation batch count 255430


  0%|          | 0/255430 [00:00<?, ?it/s]

{'accuracy': 0.9001763209265975}
{'f1': 0.8988117826633272}
	 Training. Acc: 90.02%
